# Encode Datasets

This notebook takes as input a trained encoder, target set, and query set, and saves encoded targets & queries, as well the query-target distance matrix.

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

import primo.models
import primo.datasets

from tqdm.notebook import tqdm

In [ ]:
encoder = primo.models.Encoder('/tf/primo/data/models/encoder-model.h5')

### Encode Queries
This code loads the query features, encodes them to DNA sequences, and saves the result.

In [ ]:
query_features = pd.read_hdf('/tf/primo/data/queries/features.h5')

In [ ]:
query_seqs = encoder.encode_feature_seqs(query_features)
pd.DataFrame(
    query_seqs, index=query_features.index, columns=['FeatureSequence']
).to_hdf(
    '/tf/primo/data/queries/feature_seqs.h5', key='df', mode='w'
)

### Encode Target Set
This code loads the target set's features, encodes them to DNA sequences, calculates distances to each query, and saves the result.

In [ ]:
dist_store = pd.HDFStore('/tf/primo/data/targets/query_target_dists.h5', complevel=9, mode='w')
seq_store = pd.HDFStore('/tf/primo/data/targets/feature_seqs.h5', complevel=9, mode='w')

try:
    prefixes = [ "%x"%i for i in range(16) ]
    for prefix in tqdm(prefixes):
        target_features = pd.read_hdf('/tf/open_images/targets/features/targets_%s.h5' % prefix)

        distances = {}
        for query_id, query in query_features.iterrows():
            distances[query_id] = np.sqrt(np.square(target_features.values - query.values).sum(1))

        df = pd.DataFrame(distances, index=target_features.index)
        dist_store.append('df', df)
        del df, distances
        
        target_seqs = encoder.encode_feature_seqs(target_features)
        df = pd.DataFrame(target_seqs, index=target_features.index, columns=['FeatureSequence'])
        seq_store.append('df', df)
        del df, target_seqs
        
        del target_features

finally:
    dist_store.close()
    seq_store.close()

### Encode Extended Target Set
This code loads the extended target set's features, encodes them to DNA sequences, calculates distances to each query, and saves the result.

In [ ]:
dist_store = pd.HDFStore('/tf/primo/data/extended_targets/query_target_dists.h5', complevel=9, mode='w')
seq_store = pd.HDFStore('/tf/primo/data/extended_targets/feature_seqs.h5', complevel=9, mode='w')

try:
    prefixes = [ "%x"%i for i in range(16) ]
    for prefix in tqdm(prefixes):
        target_features = pd.read_hdf('/tf/open_images/extended_targets/features/extended_targets_%s.h5' % prefix)

        distances = {}
        for query_id, query in query_features.iterrows():
            distances[query_id] = np.sqrt(np.square(target_features.values - query.values).sum(1))

        df = pd.DataFrame(distances, index=target_features.index)
        dist_store.append('df', df)
        del df, distances
        
        target_seqs = encoder.encode_feature_seqs(target_features)
        df = pd.DataFrame(target_seqs, index=target_features.index, columns=['FeatureSequence'])
        seq_store.append('df', df)
        del df, target_seqs
        
        del target_features

finally:
    dist_store.close()
    seq_store.close()